# SymbolicAI: Basics

### Get Imports

In [5]:
import os
import warnings
warnings.filterwarnings('ignore')
os.chdir('../') # set the working directory to the root of the project
import numpy as np
from symai import *
from symai.components import *
from IPython.display import display

### Using Symbols

In [2]:
sym = Symbol('Hi there!')
sym

<class 'symai.symbol.Symbol'>(value=Hi there!)

Adding two symbols together.

In [3]:
res = sym + 'how are you?'
res

<class 'symai.symbol.Symbol'>(value=Hi there! How are you?)

If one of the objects is a symbol then the other object is converted to a symbol.

A `Symbol` instance takes in any type of object and preserves its original value type until it is evaluated and sent to the neuro-symbolic engine:

In [6]:
sym = Symbol(np.array([5, 2, 42, 1]))
sym.value_type

numpy.ndarray

One can also easily retriev the object by accessing the `value` attribute:

In [7]:
sym.value

array([ 5,  2, 42,  1])

We can add multiple values together and their operation will be contextualized evaluated. Here we use the shift operator to integrate a new value into the existing symbol:

In [8]:
res = sym << 2
res

<class 'symai.symbol.Symbol'>(value=['5', '2', '42', '1', '2'])

We could do the same manipulation with the `+` operator, however, the prompts designed for each individual operator may contain different information, therefore, therefore may not evaluated to similar outcomes. In the following example it is clear that the `+` operator is not to shift the value into the array:

In [9]:
res = sym + 2
res

<class 'symai.symbol.Symbol'>(value=["5", "2", "42", "3"])

However, if we use the `+` operator to evaluate arithmetic operations, the results are quite different:

In [10]:
x = Symbol('x')
x += 1
x

<class 'symai.symbol.Symbol'>(value=x + 1)

For simple expressions and operations we can use LLMs to interpret the mathematical expression and evaluate the result:

In [4]:
x = Symbol('x + 1 = 0')
x.interpret()

<class symai.symbol.Symbol at 0x16f4b4610>(value=x + 1 = 0

To solve for x:

x + 1 - 1 = 0 - 1

x = -1)

### Evaluating Expressions

Similar to `word2vec` we intend to preform contextualized operations on different symbols. 

Word2vec is a machine learning algorithm that is used to generate dense vector representations of words. It works by training a shallow neural network to predict a word given its neighbors in a text corpus. The resulting vectors are then used in a wide range of natural language processing applications, such as sentiment analysis, text classification, and clustering.

Below we can see an example how one can perform operations on the word embedding vectors (colored boxes).

<img src="../assets/images/img3.png" width="470px">

However, we can also perform operations on the natural language space itself. For example, we can add two the word expressions similar to how we added them in the word2vec space:

In [3]:
Symbol('King - Man + Women').interpret()

<class symai.symbol.Symbol at 0x16e47da10>(value=Queen)

As we can see, if the `expression` method is empty the default implementation of `Symbol` uses only the `value` stored within the current symbol. 

Now let's try to evaluate a more complex expression:

In [2]:
expression = Interface('wolframalpha')
res = expression('x^2 + 2x + 1')
res


<class 'symai.symbol.Symbol'>(value=x^2 + 2x + 1)

Well, not exactly simplified the way we intended it. 

This is because using LLMs for expression evaluations is still very limited, therefore, to leverage the full power of symbolic programming we can use the `expression` method to evaluate an expression via `WolframAlpha`:

In [12]:
sym = Symbol()
expression = Interface('wolframalpha')
res = expression('x^2 + 2x + 1')
res

<class 'symai.symbol.Expression'>(value=x = -1)

There we go! The `command` method configures the `symbolic` engine to evaluate expressions via `WolframAlpha`. One can see that this evaluates the expression and returns the correct result.

### Showing other basic operations

Sometimes we want to simply concatenate to symbols together without any other neural operation. This is easily done with the `|` operation:

In [5]:
# define a second string
sym = Symbol("Welcome to our tutorial.")
test2 = 'Hello world!'
# concatenate strings
res = sym | test2
res

<class symai.symbol.Symbol at 0x16ffcf890>(value=Welcome to our tutorial.Hello world!)

We can of course define full sentences as Symbols and perform several operations on them.

In [6]:
sym = Symbol("""Images and videos of domestic cats make up some of the most viewed content on the World Wide Web, particularly image macros in the form of lolcats. ThoughtCatalog has described cats as the "unofficial mascot of the Internet".[1]
The subject has attracted the attention of various scholars and critics, who have analyzed why this form of low art has reached iconic status. Although it may be considered frivolous, cat-related Internet content contributes to how people interact with media and culture.[2] Some argue that there is a depth and complexity to this seemingly simple content, with a suggestion that the positive psychological effects that pets have on their owners also hold true for cat images viewed online.[3]
Research has suggested that viewing online cat media is related to positive emotions, and that it even may work as a form of digital therapy or stress relief for some users. Some elements of research also shows that feelings of guilt when postponing tasks can be reduced by viewing cat content.[4]
""")

Here we translate the existing Symbol to German:

In [7]:
sym.translate('German')

<class symai.symbol.Symbol at 0x10dc856d0>(value=Bilder und Videos von Hauskatzen gehören zu den am meisten gesehenen Inhalten im World Wide Web, insbesondere Bildmakros in Form von Lolcats. ThoughtCatalog hat Katzen als das "inoffizielle Maskottchen des Internets" beschrieben.[1]
Das Thema hat die Aufmerksamkeit verschiedener Gelehrter und Kritiker auf sich gezogen, die analysiert haben, warum diese Form der niedrigen Kunst einen ikonischen Status erreicht hat. Obwohl es als belanglos angesehen werden kann, trägt internetbezogener Katzeninhalt dazu bei, wie Menschen mit Medien und Kultur interagieren.[2] Einige argumentieren, dass es eine Tiefe und Komplexität in diesem scheinbar einfachen Inhalt gibt, mit dem Vorschlag, dass die positiven psychologischen Effekte, die Haustiere auf ihre Besitzer haben, auch für online betrachtete Katzenbilder zutreffen.[3]
Die Forschung hat vorgeschlagen, dass das Anschauen von Katzenmedien im Internet mit positiven Emotionen verbunden ist und dass es sogar als eine Form der digitalen Therapie oder Stressentlastung für einige Nutzer dienen kann. Einige Forschungsergebnisse zeigen auch, dass Gefühle der Schuld, wenn Aufgaben aufgeschoben werden, durch das Betrachten von Katzeninhalten reduziert werden können.[4])

And now, we try to classify how the mood of the above sentences:

In [8]:
sym.choice(['angry', 'neutral', 'hate-speech', 'happy', 'unk'], default='unk')

<class symai.symbol.Symbol at 0x16ffc5c50>(value=neutral)

## Documentation Generator Example

In [9]:
from examples.docs import Docs, CppDocs
docs = Docs()

In [10]:
doc = docs("""def execute(default: str = None,
            constraints: List[Callable] = [],
            pre_processors: List[PreProcessor] = [],
            post_processors: List[PostProcessor] = [],
            **decorator_kwargs):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(instance, *args, **kwargs):
            return execute_func(instance,
                                func=func,
                                constraints=constraints,
                                default=default,
                                pre_processors=pre_processors,
                                post_processors=post_processors,
                                **decorator_kwargs)
        return wrapper
    return decorator
""")
print(doc)

"""Decorates a function to be executed with pre_processors, post_processors, and constraints.

This decorator is used to apply preprocessing, postprocessing, and constraint validation to the output of a function. It provides a way to hook into the execution of the function, allowing for additional processing or validation steps to be applied before and after the function runs.

Args:
    default (str, optional): The default value to be returned if the validation of constraints fails. It can also be used to provide a default response if the function cannot be executed. Defaults to None.
    constraints (List[Callable], optional): A list of callables that are used to validate the output of the function. An empty list means no constraints will be applied. Defaults to an empty list ( [] ).
    pre_processors (List[PreProcessor], optional): A list of preprocessing objects that are applied to the arguments of the function before the function is called. Preprocessors can modify or augment the

## Markdown

We can actually on the fly also generate markdown documentation for our notebooks. This is done by using the built in `query` method. The `query` method is used to query the `symbolic` engine for information about the current symbol. The `query` method can therefore be used to ask the engine to generate documentation in a specific format.

In [11]:
res = doc.query(context="Write a very detailed Markdown documentation the News class.")
print(res)

```markdown
# News Class Documentation

The `News` class is a utility that helps developers inject uniform preprocessing, postprocessing, and constraint validation behaviors into their methods. It's implemented as a Python decorator, which can be easily applied to any function.

## Overview

The `News` class decorator streamlines the process of enhancing functions with additional processing layers. It allows functions to be wrapped such that:

- Input arguments can be pre-processed before the function runs.
- Output can be post-processed after the function completes.
- The output can be validated against constraints to ensure it adheres to certain requirements.

By using this decorator, developers can maintain cleaner codebases by separating concerns and avoiding repetitive code.

## Features

### Pre-Processing

Pre-processors are used to transform or prepare the input data before the core logic of the function is executed. They can be used, for example, to validate input types, conve